In [1]:
# uncomment and run if mismo is not installed
# %pip install -q git+https://github.com/NickCrews/mismo@main

In [2]:
from __future__ import annotations

import ibis
from ibis import _
import ibis.expr.types as it

ibis.options.interactive = True

We are going to dedupe the PATSTAT dataset. Every record represents a patent filed,
and our task is to determine which one came from the same inventor.
The end goal is to add a column called `inventor_id` to each patent.
This dataset contains a ground truth label, so we can evaluate how well we did.

`Mismo` contains this as an included dataset so it is easy to get started.
The returned dataset is an Ibis table, which is a lazy representation of a SQL table.
It is similar to a pandas dataframe, but has a few properties that make it much
better for the record linkage use case:

- Since it is SQL backed, it can handle datasets that are larger than memory, in
  the many millions of rows.
- Computation is performed by the powerful SQL backend of your choice: Google BigQuery,
  Apache Spark, Snowflake, etc. For this demo, we use DuckDB, which is a 
  state-of-the-art SQL engine based around a columnar data model
  (ie oriented towards the bulk operations of record linkage)
- Ibis is strongly typed, has a full API, is well-documented, and has good
  integration with the rest of the python data science ecosystem.


In [3]:
from mismo.datasets import load_patents  # noqa: E402

patents = load_patents()
print(patents.count())
patents

2379


┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │     0.00 │  0.000000 │ GABRIEL MARINUS  MEESTERS**RUDOLF CAROLUS  BARENDSE**ARIE KARST  KIES**ALEXANDE… │ C12N**A61K**A23L**A23J**A23K**A01H**B01J**C12R**C07D**A61P**B01D │
│         … │          … │ …                    │ …                            │        … │         … │ …                                                                                │ …  

In [4]:
from mismo import clean  # noqa: E402

clean.distribution_dashboard(patents, column="name")

Let's clean this up a bit:
- clean up whitespace
- convert the `classes` column to actual arrays (they really represent sets).
  Each element in `classes` is a 4-character IPC technical code that is like a tag
  for the patent. Similar patents will have similar tags.
- convert `coauthors` into a sorted list of tokens.

Of course, this sort of cleaning only happens after a lot of exploring the
data manually. This isn't anything in particular to record linkage,
but this needs to happen for any sort of data analysis.


In [5]:
def clean_names(names: it.StringValue) -> it.StringValue:
    names = clean.norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Some company names have "NV" or "BV" in them, which is a Dutch
    # classification of businesses sort of like LLC vs Corporation in the US.
    # These seem to just be noise and don't help us distinguish between
    # different companies, so we want to remove them.
    # However, sometimes the names are ill-formed or have other junk in them
    # like "philipsN V" or "philipsN.V."
    names = names.re_replace(r"B\s*V", "")
    names = names.re_replace(r"N\s*V", "")
    # Also remove other common stopwords that usually just add noise
    names = names.re_replace(r"\bCO\b", "")
    names = names.re_replace(r"\bCORP\b", "")
    names = names.re_replace(r"\bLTD\b", "")
    names = names.re_replace(r"\bINC\b", "")
    names = names.re_replace(r"\bLLC\b", "")
    names = names.re_replace(r"\bNL\b", "")
    # Now have to do whitespace fixup again
    names = clean.norm_whitespace(names)
    return names


def clean_coauthors(s: it.StringValue) -> it.ArrayValue:
    """Turn a 'list' of coauthors into a sorted array of cleaned name tokens.

    The raw string looks like:
    "DAVID E  SNYDER**THOMAS D  LYSTER"
    With "**" as the separator. However, entire names are not always useful
    to differentiate between people: there are some records where a coauthor is
    listed as "ALKEMA DUURT PIETER WILLEM" and another as "ALKEMA DUURT".
    These are probably the same person, but if we compared entire names with each other
    that would be a false negative. So instead we turn this into a sorted array of
    name tokens, eg for the example above:
    ["DAVID", "E", "SNYDER", "THOMAS", "D", "LYSTER"].

    Except we also want to remove initials and other stopwords like "VAN"
    so scrub all tokens less than 3 characters long.
    """
    s = s.upper().nullif("NONE").replace("**", " ")
    s = clean.norm_whitespace(s)
    return s.split(" ").unique().sort().filter(lambda x: x.length() >= 3)


def clean_classes(s: it.StringValue) -> it.ArrayValue:
    return s.upper().split("**").map(clean.norm_whitespace).sort()


cleaned = patents.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=clean_names(patents.name),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=clean_coauthors(patents.coauthors),
    classes=clean_classes(patents.classes),
)

Then, let's add some features. The binned coordinates will be used in the blocking step,
so that locations in the same lat/lng bin will be compared to each other.

We also generate some features based on the `name` column.

In [6]:
def bin_lat_lon(lat: it.FloatingValue, lon: it.FloatingValue) -> it.StructValue:
    """Bin a latitude or longitude to 0.1 degree precision, which is ~6 miles.

    If both are null, return null.

    (52.35, 4.916667 -> (524, 49)
    """

    def _bin_coord(coord: it.FloatingValue) -> it.IntegerValue:
        return (coord.round(1) * 10).cast("int16").fillna(0)

    result = ibis.struct(
        {
            "lat_hash": _bin_coord(lat),
            "lon_hash": _bin_coord(lon),
        }
    )
    both_null = lat.isnull() & lon.isnull()
    return both_null.ifelse(ibis.null(), result)


featured = cleaned.mutate(
    name_tokens=cleaned.name_cleaned.split(" ").map(clean.norm_whitespace).sort(),
    name_first3=_.name_cleaned[0:3],
    coords_hashed=bin_lat_lon(_.latitude, _.longitude),
)
featured

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ name_cleaned         ┃ latitude ┃ longitude ┃ coauthors                         ┃ classes                  ┃ name_tokens                 ┃ name_first3 ┃ coords_hashed                            ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ string               │ float64  │ float64   │ array<string>                     │ array<string>            │ array<string>               │ string      │ struct<lat_hash: int16, lon_hash: int16> │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────────────────┼──────────┼───────────┼───────────────────────────────────┼──────────────────────────┼─────────────────────────────┼─────────────┼──────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │ AGILENT TECHNOLOGIES │     NULL │      NULL │ ['DAVID', 'ELECTRONICS', ... +5]  │ ['A61B', 'A61N']         │ ['AGILENT', 'TECHNOLOGIES'] │ AGI         │ NULL                                     │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │ AKZO NOBEL           │     NULL │      NULL │ ['ALBERT', 'ALBERTO', ... +31]    │ ['B01L', 'C11D', ... +3] │ ['AKZO', 'NOBEL']           │ AKZ         │ NULL                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │ AKZO NOBEL           │     NULL │      NULL │ ['CHRISTINA', 'ERNEST', ... +11]  │ ['B01F', 'C09K', ... +2] │ ['AKZO', 'NOBEL']           │ AKZ         │ NULL                                     │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │    52.35 │  4.916667 │ ['EVAN', 'GRANT', ... +8]         │ ['G02B', 'G04G', ... +2] │ ['ALCATEL']                 │ ALC         │ {'lat_hash': 524, 'lon_hash': 49}        │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │    52.35 │  4.916667 │ ['CHRISTIAN', 'DUANE', ... +7]    │ ['B05D', 'C03B', ... +6] │ ['ALCATEL']                 │ ALC         │ {'lat_hash': 524, 'lon_hash': 49}        │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │     NULL │      NULL │ ['AUDOUIN', 'GABRIAGUES', ... +4] │ ['H01S', 'H04B', ... +1] │ ['ALCATEL']                 │ ALC         │ NULL                                     │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │ CANON EUROPA         │     NULL │      NULL │ ['CANON', 'CENT', ... +11]        │ ['G06F']                 │ ['CANON', 'EUROPA']         │ CAN         │ NULL                                     │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │ CANON EUROPA         │     NULL │      NULL │ ['ADAM', 'ALEXANDER', ... +5]     │ ['G01B', 'G06T']         │ ['CANON', 'EUROPA']         │ CAN         │ NULL                                     │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │ CANON EUROPA         │     NULL │      NULL │ ['BARRY', 'BROCARD', ... +24]     │ ['B41J', 'G01B', ... +8] │ ['CANON', 'EUROPA']         │ CAN         │ NULL                                     │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │ DSM                  │     NULL │      NU

OK, now it's time to block! This is where we generate comparisons between records.
If we were naive and generated all possible comparisons from N record,
you would end up with N^2 comparisons. For our small dataset of ~2000 records
we would be able to get away with this, but for datasets much larger than this
it would be infeasible.

In [7]:
from mismo.block import BlockingRule, BlockingRules  # noqa: E402

rules = BlockingRules(
    BlockingRule("Coordinates Close", "coords_hashed"),
    BlockingRule("Name First 3", "name_first3"),
    BlockingRule("Coauthors Exact", "coauthors"),
    BlockingRule("Classes Exact", "classes"),
)

featured = featured.cache()
blocked = rules.block(featured, featured, labels=True)
blocked = blocked.cache()
blocked

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ blocking_rules                               ┃ classes_l                ┃ classes_r                ┃ coauthors_l               ┃ coauthors_r               ┃ coords_hashed_l                          ┃ coords_hashed_r                          ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                  ┃ name_cleaned_r                  ┃ name_first3_l ┃ name_first3_r ┃ name_l                                ┃ name_r                                ┃ name_tokens_l                          ┃ name_tokens_r                          ┃ name_true_l                ┃ name_true_r                ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>                                │ array<string>            │ array<string>            │ array<string>             │ array<string>             │ struct<lat_hash: int16, lon_hash: int16> │ struct<lat_hash: int16, lon_hash: int16> │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string                          │ string                          │ string        │ string        │ string                                │ string                                │ array<string>                          │ array<string>                          │ string                     │ string                     │
├─────────────┼─────────────┼──────────────────────────────────────────────┼──────────────────────────┼──────────────────────────┼───────────────────────────┼───────────────────────────┼──────────────────────────────────────────┼──────────────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼─────────────────────────────────┼─────────────────────────────────┼───────────────┼───────────────┼───────────────────────────────────────┼───────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────┼────────────────────────────┤
│      662355 │      662355 │ ['Coauthors Exact', 'Classes Exact', ... +1] │ ['C03B', 'C03C', ... +2] │ ['C03B', 'C03C', ... +2] │ ['WEBER']                 │ ['WEBER']                 │ NULL                                     │ NULL                                     │       656303 │       656303 │       NULL │       NULL │        NULL │        NULL │ ALCATEL AG                      │ ALCATEL AG                      │ ALC           │ ALC           │ ALCATEL AG                            │ ALCATEL AG                            │ ['AG', 'ALCATEL']                      │ ['AG', 'ALCATEL']                    

The result of that was the two tables joined together, with a `_l` added
to all the columns from the left table, and a `_r` added to all the columns
from the right table. In addition, there is a column `blocking_rules` that
tells us which blocking rules were used to generate the pair.

By blocking, we reduced the number of needed pairs by a large factor.
In larger datasets, and with better blocking rules, this would be even more!


In [8]:
from mismo import metrics  # noqa: E402

n_comparisons = blocked.count().execute()
n_naive = metrics.n_naive_comparisons(featured)
reduction_ratio = n_comparisons / n_naive
n_naive, n_comparisons, reduction_ratio

(2828631, 643357, 0.22744465432217917)

We can also inspect which blocking rules were most to blame for the generated
pairs. If some rules generate a huge amount of comparisons, it might be worth
trying to make them more restrictive so we get better performance. Or, if some
blocking rules aren't generating any comparisons, that might be an indication
that we have a bug in there somewhere.

In [9]:
from mismo.block import upset_chart  # noqa: E402

upset_chart(blocked)

alt.VConcatChart(...)

OK, now that we have our candidate pairs generated, let's actually do the
comparing of pairs. There are many ways to do this, but one of the most common
is to generate a set of Comparison objects, each of which represents a
measurement of similarity based on some dimension (eg "location", "name", "price", etc).
Each Comparison is composed of Levels, which represent discrete levels of aggreement
(eg "exactly", "within 100km", "one or both values null", etc)

In [10]:
from mismo.compare import (  # noqa: E402
    Comparison,
    ComparisonLevel,
    Comparisons,
    distance_km,
    jaccard,
)

name_comparison = Comparison(
    name="Name",
    levels=[
        ComparisonLevel("exact", _.name_cleaned_l == _.name_cleaned_r),
        ComparisonLevel(
            "share 1 token",
            condition=_.name_tokens_l.intersect(_.name_tokens_r).length() == 1,
        ),
        ComparisonLevel(
            "share 2+ tokens",
            condition=_.name_tokens_l.intersect(_.name_tokens_r).length() >= 2,
        ),
    ],
)

classes_comparison = Comparison(
    name="Classes",
    levels=[
        ComparisonLevel("exact", _.classes_l == _.classes_r),
        ComparisonLevel(
            name="share 1 class",
            condition=_.classes_l.intersect(_.classes_r).length() == 1,
        ),
        ComparisonLevel(
            name="share 2+ classes",
            condition=_.classes_l.intersect(_.classes_r).length() >= 2,
        ),
    ],
)

coauthors_comparison = Comparison(
    name="Coauthors",
    levels=[
        # ComparisonLevel("exact", _.coauthors_l == _.coauthors_r),
        ComparisonLevel(
            name="jaccard >= 0.5",
            condition=jaccard(_.coauthors_l, _.coauthors_r) >= 0.5,
        ),
        ComparisonLevel(
            name="jaccard >= 0.25",
            condition=jaccard(_.coauthors_l, _.coauthors_r) >= 0.25,
        ),
        ComparisonLevel(
            name="jaccard >= 0.1",
            condition=jaccard(_.coauthors_l, _.coauthors_r) >= 0.1,
        ),
        ComparisonLevel(
            name="jaccard >= 0.02",
            condition=jaccard(_.coauthors_l, _.coauthors_r) >= 0.02,
        ),
    ],
)

coords_comparison = Comparison(
    name="Coords",
    levels=[
        ComparisonLevel(
            name="exact",
            condition=(_.latitude_l == _.latitude_r) & (_.longitude_l == _.longitude_r),
        ),
        ComparisonLevel(
            name="coords <= 10km",
            condition=distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 10,
        ),
        ComparisonLevel(
            name="coords <= 100km",
            condition=distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 100,
        ),
        ComparisonLevel(
            name="both coord missing",
            condition=_.coords_hashed_l.isnull() & _.coords_hashed_r.isnull(),
        ),
        ComparisonLevel(
            name="one coord missing",
            condition=_.coords_hashed_l.isnull() | _.coords_hashed_r.isnull(),
        ),
    ],
)

comparisons = Comparisons(
    [
        name_comparison,
        classes_comparison,
        coauthors_comparison,
        coords_comparison,
    ]
)
compared = comparisons.label_pairs(blocked)
compared = compared.cache()
compared

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ Name   ┃ Classes ┃ Coauthors      ┃ Coords             ┃ blocking_rules                               ┃ classes_l                ┃ classes_r                ┃ coauthors_l               ┃ coauthors_r               ┃ coords_hashed_l                          ┃ coords_hashed_r                          ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                  ┃ name_cleaned_r                  ┃ name_first3_l ┃ name_first3_r ┃ name_l                                ┃ name_r                                ┃ name_tokens_l                          ┃ name_tokens_r                          ┃ name_true_l                ┃ name_true_r                ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ string │ string  │ string         │ string             │ array<string>                                │ array<string>            │ array<string>            │ array<string>             │ array<string>             │ struct<lat_hash: int16, lon_hash: int16> │ struct<lat_hash: int16, lon_hash: int16> │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string                          │ string                          │ string        │ string        │ string                                │ string                                │ array<string>                          │ array<string>                          │ string                     │ string                     │
├─────────────┼─────────────┼────────┼─────────┼────────────────┼────────────────────┼──────────────────────────────────────────────┼──────────────────────────┼──────────────────────────┼───────────────────────────┼───────────────────────────┼──────────────────────────────────────────┼──────────────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼─────────────────────────────────┼─────────────────────────────────┼───────────────┼───────────────┼───────────────────────────────────────┼───────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────┼────────────────────────────┤
│      662355 │      662355 │ exact  │ exact   │ jaccard >= 0.5 │ both coord missing │ ['Coauthors Exact', 'Classes Exact', ... +1] │ ['C03B', 'C03C', ... +2] │ ['C03B', 'C03C', ... +2] │ ['WEBER']                 │ ['WEBER']                 │ NULL                                     │ NULL                                     │ 

The result above is the blocked table, with a column added for every `Comparison`.
The value of each column is the level that the record pair matched at.
For example, there is now a "Name" column,
filled with values like "exact", "share 1 token", etc.

Now that we have our features, we can use the Fellegi-Sunter model to train weights
for each of these features. This is a probabilistic model that is based on the concept
of odds. When you see an exact match on name, that increases the odds of a match
by some amount, maybe 10x. When you see a non-match on name, that decreases the odds
of a match by some amount, maybe 0.1x. We can either train this from labeled data,
or we can use unlabeled data using an algorithm called "Expectation Maximization".

In [11]:
from mismo.fs import train_comparisons_using_em  # noqa: E402

weights = train_comparisons_using_em(
    comparisons, featured, featured, max_pairs=100_000, seed=42
)
# Can save and load weights
# weights.to_json("weights.json")
# weights = ComparisonWeights.from_json("weights.json")
weights.plot()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.VConcatChart(...)

In the above plot, you can see the odds for "share 1 class" and "share 2+ classes"
are nearly the same. That doesn't quite make sense, and might be an indicator
that we should look more into this to see if there is a problem with the features
that we are using. But for now let's just move on.

Use the weights to score the record pairs, finding the odds for each
`Comparison``, and then combining them into an overall odds for the record pair.

In [12]:
scored = weights.score(compared)
scored = scored.cache()
scored

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ odds         ┃ Name   ┃ Name_odds ┃ Classes ┃ Classes_odds ┃ Coauthors      ┃ Coauthors_odds ┃ Coords             ┃ Coords_odds ┃ blocking_rules                               ┃ classes_l                ┃ classes_r                ┃ coauthors_l               ┃ coauthors_r               ┃ coords_hashed_l                          ┃ coords_hashed_r                          ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                  ┃ name_cleaned_r                  ┃ name_first3_l ┃ name_first3_r ┃ name_l                                ┃ name_r                                ┃ name_tokens_l                          ┃ name_tokens_r                          ┃ name_true_l                ┃ name_true_r                ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ float64      │ string │ float64   │ string  │ float64      │ string         │ float64        │ string             │ float64     │ array<string>                                │ array<string>            │ array<string>            │ array<string>             │ array<string>             │ struct<lat_hash: int16, lon_hash: int16> │ struct<lat_hash: int16, lon_hash: int16> │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string                          │ string                          │ string        │ string        │ string                                │ string                                │ array<string>                          │ array<string>                          │ string                     │ string                     │
├─────────────┼─────────────┼──────────────┼────────┼───────────┼─────────┼──────────────┼────────────────┼────────────────┼────────────────────┼─────────────┼──────────────────────────────────────────────┼──────────────────────────┼──────────────────────────┼───────────────────────────┼───────────────────────────┼──────────────────────────────────────────┼──────────────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼─────────────────────────────────┼─────────────────────────────────┼───────────────┼───────────────┼───────────────────────────────────────┼───────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────────────────┼────────────────────────

We can plot these compared pairs.
We can see which comparison levels are most common,
which occur together,
which lead to matches, and which lead to non-matches.

The exact match levels have the highest odds, and the
else levels have the lowest. The other levels are somewhere in between.

In [13]:
from mismo.compare import compared_dashboard  # noqa: E402

compared_dashboard(compared, comparisons=comparisons, weights=weights)

It looks like an odds of 50 seems to separate the pairs between non-matches
and matches.
If I hover over the above chart, I can see that pretty much all the "else" comparisons
are in the low cluster, and all the "exact" comparisons are in the high cluster.

In [14]:
odds_threshold = 50
(scored.odds >= odds_threshold).value_counts()

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ GreaterEqual(odds, 50) ┃ GreaterEqual(odds, 50)_count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ boolean                │ int64                        │
├────────────────────────┼──────────────────────────────┤
│ False                  │                       403910 │
│ True                   │                       239447 │
└────────────────────────┴──────────────────────────────┘

Let's be really picky and only take the most likely matches as true matches, and
then perform connected components to label each patent with its inventor:

In [15]:
from mismo.cluster import connected_components  # noqa: E402

links = scored[_.odds >= odds_threshold]
links = links.cache()
print(links.count().execute())
labels = connected_components(links, nodes=featured.record_id)
print(labels.count().execute())
labels

239447
2379


┏━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ record_id ┃ component ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━┩
│ int64     │ int64     │
├───────────┼───────────┤
│   8124193 │   7658815 │
│  33949163 │  33949163 │
│  22849133 │  22849133 │
│  30956705 │  30956705 │
│  10506233 │  10506233 │
│   8777330 │   8776829 │
│  12916446 │  12916446 │
│  25033386 │  23912436 │
│  35414626 │  35414624 │
│    317977 │     79860 │
│         … │         … │
└───────────┴───────────┘

In [16]:
patents_labeled = patents.left_join(labels, "record_id")
patents_labeled = patents_labeled.relocate("component", after="label_true")
patents_labeled.order_by("component").head(20)

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ component ┃ name_true            ┃ name                         ┃ latitude  ┃ longitude ┃ coauthors                                                                        ┃ classes                                                                          ┃ record_id_right ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ int64     │ string               │ string                       │ float64   │ float64   │ string                                                                           │ string                                                                           │ int64           │
├───────────┼────────────┼───────────┼──────────────────────┼──────────────────────────────┼───────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────┼─────────────────┤
│      2909 │     402600 │      2909 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │  0.000000 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                                       │            2909 │
│      3574 │     569309 │      3574 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │  0.000000 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                                     │            3574 │
│      3575 │     569309 │      3574 │ AKZO NOBEL           │ * AKZO NOBEL NV              │  0.000000 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                                           │            3575 │
│    578319 │     569309 │      3574 │ AKZO NOBEL           │ AKZO NOBEL                   │  0.000000 │  0.000000 │ MARTIN HELLSTEN**PIELOCH MARK**JOHAN ROERAADE**KARLSSON**FREDDIE HANSSON**MARCE… │ C10M**A23G**A61K**C01B**F28F**D21H**A23K**G01N**C08L**H01M**F15D**C08K**C08J**C… │          578319 │
│    578320 │     569309 │      3574 │ AKZO NOBEL           │ Akzo Nobel                   │ 52.500000 │  5.750000 │ BENNINK HERMAN JAN TIJMEN COELINGH**VERBOST PIETER M                             │ A61K                                                                             │          578320 │
│    578321 │     569309 │      3574 │ AKZO NOBEL           │ Akzo Nobel                   │ 51.983333 │  5.916667 │ ZHANG MINGIANG**GROOTENHUIS PETER**MOOREN PIETER GERARDUS**GIBSON SAMUEL GEORGE… │ C12N**A61K**C07D**C07H**C07K**A61P**C08B                                         │          578321 │
│    578322 │     569309 │      3574 │ AKZO NOBEL           │ Akzo Nobel                   │ 51.983333 │  5.916667 │ TAMMER MARINUS CATHARINUS**HOGT ANDREAS HERMAN                                   │ B01D                                                                             │          578322 │
│    578328 │     569309 │      3574 │ AKZO NOBEL           │ AKZO NOBEL B.V.              │ 51.983333 │  5.916667 │ LASCHEWSKY ANDRA**PAULKE BERND REINER**STRUCK OLIVER**FRAUNHOFER GES ZUR FORDER… │ B01D**C08F**D21H**C08K**C08L                                                     │          578328 │
│    578329 │     569309 │      3574 │ AKZO NOBEL           │ Akzo Nobel B.V.          

Now let's evaluate how good our labeling is. Mismo wraps all of the evaluation
metrics from sklearn, so we can use them with Ibis Tables.

In [17]:
labels_true = patents_labeled.select("record_id", label=_.label_true)
labels_pred = patents_labeled.select("record_id", label=_.component)
print(metrics.adjusted_rand_score(labels_true, labels_pred))
print(metrics.homogeneity_score(labels_true, labels_pred))
print(metrics.completeness_score(labels_true, labels_pred))

0.6762121486626183
0.807871573325461
0.9596503545755772
